In [1]:
import pandas as pd
import numpy as np
import glob
import math

In [2]:
bins=[float(x) for x in range(0,150,5)]

In [3]:
bins=[float(x) for x in range(0,150,2)]

In [4]:
bin_widths=np.array([bins[i]-bins[i-1] for i in range(1,len(bins))])

In [4]:
file_name="Claudia_Events/dm_dist_300_100.0_100_0.dat"

In [91]:
test_arr=pd.read_csv(file_name,sep=' ',
            header=None).values[:,2]

In [54]:
energies=test_arr[:,2]

In [55]:
vals,ebins=np.histogram(energies,bins=bins)

In [70]:
vals_normed=vals/bin_widths

In [68]:
mv,mx,dist,angle=file_name.split('_')[3:]
angle=angle.split(".dat")[0]

In [18]:
summary_file="Claudia_Events5/minos_abs_claudia_test.dat"

In [19]:
event_file_list=set(glob.glob("Claudia_Events5/minos_abs_dm_dist*"))

In [43]:
summary_file="Claudia_Events5/minos_claudia_test.dat"

event_file_list=set(glob.glob("Claudia_Events5/minos_dm_dist*"))

In [52]:
summary_file="Claudia_Events5/nova_abs_claudia_test.dat"

event_file_list=set(glob.glob("Claudia_Events5/nova_abs_dm_dist*"))

In [61]:
summary_file="Claudia_Events5/nova_claudia_test.dat"

event_file_list=set(glob.glob("Claudia_Events5/nova_dm_dist*"))

In [5]:
summary_file="Claudia_Events/OPERA_claudia_test.dat"

event_file_list=set(glob.glob("Claudia_Events/OPERA_dm_dist*"))

In [7]:
def bin_events(event_file_list,filepath,summary_file,scaling=1.0):
    #files = dict()
    #for f in event_file_list:
    #    files[float(fs.readline().split()[1])] = f
    sum_csv = pd.read_csv(summary_file,header=None,names=range(11),
                          sep=' ', skipinitialspace=True).values

    for i in xrange(len(sum_csv)):
        if sum_csv[i,0] != 'Run':
            continue
        else:
            run_name = filepath+sum_csv[i,1]
        if run_name not in event_file_list:
            continue
        sum_num=0;
        MV=0;
        MX=0;
        angle=0
        dist=0
        tot=0;
        alpha_b=0;
        for j in xrange(i,len(sum_csv)):
            if sum_csv[j,0]=="Total":
                sum_num = sum_csv[j,3]
                MV = sum_csv[j,1]
                MX = sum_csv[j,2]
                tot = sum_csv[j,8]
                alpha_b = sum_csv[j,5]
                mv,mx,dist,angle=run_name.split('_')[-4:]
                angle=angle.split(".dat")[0]
                if float(mv)/1000.0 != float(MV):
                    print("Consistency fail " + str(float(mv)/1000.0) + " " + run_name)
                #print(sum_num,MV,MX)
                break
        
        energies=pd.read_csv(run_name,header=None,
                              sep=' ',skipinitialspace=True).values[:,2]

        evals,ebins = np.histogram(energies,bins=bins)
        
        evals = evals/bin_widths
        try:
            scale = tot/float(sum_num)/scaling
        except ZeroDivisionError:
            print run_name
            scale = 1.0
        yield np.append([MV,MX,dist,angle,alpha_b],evals*scale)

In [24]:
generator = bin_events(event_file_list,"Claudia_Events/",summary_file,scaling=100)

In [25]:
outb = [x for x in generator] 

In [26]:
outb2=np.array(outb,dtype='float32')

In [27]:
titles=["M_A(GeV)","M_DM(GeV)","Distance(m)","Angle(Degrees)","alpha_B"]

In [28]:
header=np.concatenate((titles,bins[:-1]))

In [29]:
output=pd.DataFrame(data=outb,columns=header);

In [30]:
output.to_csv("OPERA_sample_distributions.csv",index=False)